In [6]:
system_prompt = """
元认知的结构如下：
    1. 元认知知识: 你对自己和他人在思考、解决问题和学习过程方面的了解。包括陈述性知识，程序性知识、条件性知识。
        1. 陈述性知识：关于自己和任务的事实，不包含任务本身的事实
        2. 程序性知识：知道如何执行策略
        3. 条件性知识：知道何时及为何使用策略
    2. 元认知调节: 你用来控制思维的活动和策略。包括规划、监控和评估。
        1. 规划：在开始任务前制定目标和选择策略。例如，决定在阅读前先浏览标题和问题以了解重点。
        2. 监控：在任务进行中检查进展和理解程度。例如，边读边问自己“我明白了吗？” 
        3. 评估：任务完成后反思结果和策略的有效性。例如，考试后回顾哪些复习方法最有用。
    3. 元认知体验: 学习新知识或尝试解决问题时产生的想法和感受。以下是其主要类型：
        1. 知道感（Feeling of Knowing, FOK）  
            定义：在回忆信息前，觉得自己知道答案的一种感觉。  
            示例：考试时，看到问题后觉得答案就在嘴边，但尚未完全回忆起来。  
            作用：这种感觉可以指导是否继续努力回忆或转向其他策略。
        2. 舌尖现象（Tip-of-the-Tongue Experience, TOT）  
            定义：知道某事但暂时无法回忆起来的感觉，通常伴随部分相关信息的激活。  
            示例：试图回忆演员的名字，但只记得他们的电影角色。  
            作用：这种体验提示记忆检索的接近性，可能促使个体通过提示或上下文辅助回忆。
        3. 信心判断（Confidence Judgments）  
            定义：对自己答案或决定的确定程度评估。  
            示例：在选择题中，感觉某个选项非常正确，决定不更改。  
            作用：影响决策，如是否坚持答案或重新检查。
        4. 难度评估（Difficulty Assessments）  
            定义：感知任务的难易程度或所需努力。  
            示例：阅读复杂文章时，感觉需要更多时间理解，决定放慢速度。  
            作用：帮助调整策略，如分解任务或寻求帮助。
        5. 情绪反应（Emotional Responses）  
            定义：学习过程中伴随的情绪，如满足、挫折或焦虑。  
            示例：掌握新概念后感到满足，或因反复失败感到挫折。  
            作用：情绪反应可能触发策略调整，如因挫折而选择更简单的学习方法。
        6. 监控理解（Monitoring Comprehension）  
            定义：在阅读或听讲时检查理解程度。  
            示例：边读边问自己“我明白了吗？”发现不理解后重读。  
            作用：实时调整学习过程，确保理解。
        7.自我提问（Self-Questioning）  
            定义：通过提问检查理解或进展。  
            示例：学习后问自己“这个概念的主要点是什么？”以确认掌握。  
            作用：增强主动学习，识别知识缺口。
        8.反思性思考（Reflective Thinking）  
            定义：任务完成后反思学习过程，评估策略效果。  
            示例：考试后思考哪些复习方法有效，决定下次改进。  
            作用：为未来学习提供反馈，优化方法。

反思过程使用元认知结构分析，反思内容采用结构化思维模型
"""

In [ ]:
import os
import ollama
import json

def chat_with_ollama(user_prompt :str, system_prompt :str = None):
            # 调用Ollama处理（带异常重试机制）
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = ollama.chat(
                model='qwq',
                messages=[
                    {
                    'role': 'system',
                    'content': system_prompt
                    },
                    {
                    'role': 'user',
                    'content': user_prompt
                }],
                options={'num_predict': 9000,
                        'num_ctx': 8000,
                        'temperature': 0.9,},
            )
            print(response['message']['content'])
            return response['message']['content']
        except Exception as e:
            if attempt == max_retries - 1:
                raise RuntimeError(f"Ollama接口调用失败：{str(e)}")
            
def process_corpus_with_ollama():
    corpus_dir = "../data/knowledge/output"

    
    # 元认知自省：验证目录存在性
    if not os.path.exists(corpus_dir):
        raise FileNotFoundError(f"语料目录不存在：{corpus_dir}")
    if not os.path.isdir(corpus_dir):
        raise NotADirectoryError(f"指定路径不是目录：{corpus_dir}")

    # 循环读取文件（按文件名排序）
    file_list = sorted([
        f for f in os.listdir(corpus_dir) 
        if os.path.isfile(os.path.join(corpus_dir, f))
    ])

    
    for filename in file_list:
        outputs = []
        filepath = os.path.join(corpus_dir, filename)
        user_prompt = """下面是一份投研报告，请对报告文本进行插入式修改，添加的内容需遵循元认知结构。
具体要求如下：识别文本中的思考内容，注意务必使用基于元认知结构进行反思，插入基于元认知结构的反思内容。
在标注时，请使用<meat></meta>来插入您的注释，格式为：<meta>...</meta>。
例如，‘我需要先规划在执行’编辑为‘我需要先规划在执行<meta>元认知式反思：在规划前是否对需求足够了解？</meta>’。
注意不能省略报告原文
：\n\n"""
        # 元认知自省：异常捕获与编码处理
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                user_prompt += f"<report>{content}\n\n</report>\n\n"
        except UnicodeDecodeError:
            with open(filepath, 'r', encoding='gbk') as f:
                content = f.read().strip()
                user_prompt += f"【{filename}】\n{content}\n\n"
        for ii in range(10):
            # 采样出10个结果
            #outputs.append()
            save_content = {'raw_content': content, 'output': chat_with_ollama(user_prompt, system_prompt), 'filename': filename}
            save_output([save_content])


#把输出的结果按行写入文件
def save_output(outputs
):
    with open("../data/corpus/extrospection/output.txt", "a+", encoding='utf-8') as f:
        for output in outputs:
            # format json string to one line
            output = json.dumps(output, ensure_ascii=False)
            f.write(f"{output}\n")
            


In [20]:
result = process_corpus_with_ollama()


<think>
好的，我现在需要处理用户的请求，对这份投研报告进行插入式修改，添加基于元认知结构的反思内容。用户要求使用<meat></meta>标签，并且必须引用元认知的结构：元认知知识、调节和体验中的各个子类。

首先，我得通读整个报告，理解其内容。报告主要讲述桑伯恩地图公司的历史、发展，巴菲特的投资案例以及后续变化。接下来要识别文本中的思考内容，然后根据元认知结构添加反思注释。

用户特别强调不能省略原文，所以必须保持原样，在适当的地方插入meta标签。我需要确保每个插入的内容符合元认知的不同部分，比如陈述性知识、程序性知识等，或者在调节环节涉及规划、监控、评估，体验方面如知道感、困难评估等等。

首先分析报告中的关键点：

1. 桑伯恩地图公司的起源和成功因素（地域扩张、标准化需求）
2. 技术变革带来的威胁（记分卡技术导致业务衰退）
3. 巴菲特的投资决策：关注资产负债表中的证券组合，分拆公司结构
4. 后续发展及适应新市场

接下来，在这些部分中找到合适的插入点。例如在讨论桑伯恩的成功因素时，可能涉及元认知知识中的条件性知识（为什么当时策略有效）。在巴菲特分析财务数据时，可以引用元认知调节的规划、评估步骤。

比如：

- 当原文提到“桑伯恩地图公司的发展史非常有意思！”，这里可以插入关于陈述性知识的反思，如了解行业背景的重要性。
  
- 在讨论新技术导致业务下滑时，可能涉及困难评估（难度感知），或信心判断（投资者对未来的信心）。

当分析财务数据部分，比如注意到证券组合的价值被低估，这里需要元认知调节中的监控或评估步骤。巴菲特如何规划收购策略？这属于计划阶段的反思。

需要注意每个meta标签必须明确对应元认知结构的某一部分，并且自然融入上下文。

可能遇到的问题：确保每个注释都准确贴合元认知的结构，而不仅仅是添加思考内容。例如，在提到巴菲特注意到证券组合时，可以引用条件性知识（知道何时使用某些策略来评估资产价值），或者元认知调节中的评估阶段（回顾数据判断估值合理性）。

另外，用户提供的例子是关于规划前是否了解需求，所以类似的提问方式需要模仿，但要根据具体上下文调整。例如：

原文：“巴菲特在给股东的年度信中写道，彼时，还有业务价值5亿美元火灾保险金的保险公司仍然在使用火灾保险地图...”
插入点可以考虑元认知体验中的信心判断，即巴菲特对桑伯恩未来

KeyboardInterrupt: 